In [68]:
import os
import socket
import gymnasium as gym
from gym import spaces

import json
import time
import subprocess
import struct, sys, time
import uuid
import threading
import gzip
import numpy as np

from stable_baselines3 import PPO

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
def write_utf(s):
    encoded = s.encode("utf-8")
    length = len(encoded)
    return length.to_bytes(2, byteorder="big") + encoded

In [4]:
packet_dict = {
    "PacketPing": 1,
    "PacketInitiateLogin": 2,  # Used this
    "PacketLogin": 3,          # Used this
    "PacketServerInfo": 4,
    "PacketHandshake": 5,
    "PacketKick": 6,
    "PacketQuit": 7,
    "PacketKeepConnected": 8,    # Might need this
    "PacketMessage": 9,
    "PacketPropertyColors": 10,
    "PacketCardCollectionData": 11,
    "PacketCardData": 12,
    "PacketCardActionRentData": 13,    # ACTION
    "PacketCardDescription": 14,       # ACTION
    "PacketCardPropertyData": 15,     
    "PacketCardBuildingData": 16,
    "PacketDestroyCardCollection": 17,
    "PacketDestroyCard": 18,
    "PacketPropertySetColor": 19,
    "PacketStatus": 20,
    "PacketMoveCard": 21,              # ACTION
    "PacketMovePropertySet": 22,       # ACTION?
    "PacketMoveRevealCard": 23,        # ACTION?
    "PacketMoveUnknownCard": 24,
    "PacketPlayerInfo": 25,
    "PacketPropertySetData": 26,
    "PacketUpdatePlayer": 27,
    "PacketDestroyPlayer": 28,
    "PacketRefresh": 29,
    "PacketUnknownCardCollectionData": 30,
    "PacketUndoCardStatus": 31,
    "PacketSoundData": 32,
    "PacketPlaySound": 33,
    "PacketPlayerButton": 34,
    "PacketDestroyButton": 35,
    "PacketInfoPlate": 36,
    "PacketDestroyInfoPlate": 37,
    "PacketCardButtons": 38,
    "PacketTurnOrder": 39,
    "PacketGameRules": 40,
    "PacketSetChatOpen": 41,
    "PacketRemoveMessageCategory": 42,
    "PacketSelectCardCombo": 43,           # ACTION
    "PacketSetAwaitingResponse": 44,
    "PacketChat": 45,
    "PacketActionAccept": 46,
    "PacketActionDraw": 47,                 # ACTION
    "PacketActionEndTurn": 48,              # ACTION
    "PacketActionMoveProperty": 49,         # ACTION
    "PacketActionChangeSetColor": 50,       # ACTION
    "PacketActionPay": 51,                  # ACTION
    "PacketActionPlayCardBuilding": 52,     # ACTION
    "PacketActionDiscard": 53,              # ACTION
    "PacketActionSelectPlayer": 54,         
    "PacketActionSelectProperties": 55,       
    "PacketActionSelectPlayerMonopoly": 56,
    "PacketActionUndoCard": 57,
    "PacketActionClickLink": 58,
    "PacketActionButtonClick": 59,
    "PacketActionUseCardButton": 60,
    "PacketActionRemoveBuilding": 61,           # ACTION
    "PacketActionSelectCardCombo": 62,          # ACTION
    "PacketActionMoveHandCard": 63,             # ACTION
    "PacketSoundCache": 64,
    "PacketActionStatePlayerTurn": 65,
    "PacketActionStateBasic": 66,
    "PacketActionStateRent": 67,
    "PacketActionStatePropertiesSelected": 68,
    "PacketActionStatePropertySetTargeted": 69,
    "PacketUpdateActionStateTarget": 70
}

In [5]:
def send_keep_alive(sock):
    """Send keep-alive packet to prevent timeout"""
    # PacketKeepConnected is likely ID 7 (check your NetHandler order)
    KEEP_CONNECTED_ID = 8
    
    header = struct.pack(">h", 8) + struct.pack(">i", 0)  # No payload
    sock.sendall(header)
    # print("Sent keep-alive packet")

In [6]:
def start_keep_alive_loop(sock, interval=20):
    """Send keep-alive packets regularly"""
    while True:
        time.sleep(interval)
        try:
            send_keep_alive(sock)
        except:
            break  # Stop if connection is lost

In [35]:
class MDClient:
    def __init__(self, host, port, player_name, version, protocol_version):
        self.host = host
        self.port = port
        self.player_name = player_name
        self.version = version
        self.protocol_version = protocol_version

        self.sock = socket.create_connection((self.host, self.port))
        self._login()

    
    def _login(self):
        # build payload: single int (protocol_version)
        payload = struct.pack(">i", self.protocol_version)
        payload_len = len(payload)
        
        # Header as separate fields, not combined struct
        # Packet ID is 2
        header = struct.pack(">h", packet_dict["PacketInitiateLogin"]) + struct.pack(">i", payload_len)
        msg = header + payload
        
        self.sock.sendall(msg)
        print(f"Sent PacketInitiateLogin({self.protocol_version})")
        
        # try to read a response header (2 bytes id + 4 bytes len)
        hdr = self.sock.recv(6)
        if len(hdr) < 6:
            print("No response header received (got bytes):", hdr)
        else:
            pkt_id, pkt_len = struct.unpack(">hI", hdr)
            print("Received packet id:", pkt_id, "payload length:", pkt_len)
            if pkt_len:
                data = b''
                while len(data) < pkt_len:
                    chunk = self.sock.recv(pkt_len - len(data))
                    if not chunk:
                        break
                    data += chunk
                print("Payload bytes:", data)        
        # wait for response
        # send PacketLogin (ID=3)
        UUID_BYTES = uuid.uuid4().bytes
        
        # 1. protocol_version (int)
        proto_bytes = struct.pack(">i", self.protocol_version)
        
        # 2. clientVersion (String) - FIXED: int length prefix + UTF-16BE chars
        client_version_chars = self.version.encode('utf-16be')
        client_version_payload = struct.pack(">i", len(self.version)) + client_version_chars  # int length, not short
        
        # 3. id (byte[] - 16 bytes)
        id_payload = struct.pack(">i", 16) + UUID_BYTES  # Length prefix (16) + 16 bytes of UUID
        
        # 4. name (String) - FIXED: int length prefix + UTF-16BE chars  
        name_chars = self.player_name.encode('utf-16be')
        name_payload = struct.pack(">i", len(self.player_name)) + name_chars  # int length, not short
        
        # Combine all fields
        payload = proto_bytes + client_version_payload + id_payload + name_payload
        
        # Header: packet ID (short) + payload length (int)
        # Packet ID is 3
        header = struct.pack(">h", packet_dict["PacketLogin"]) + struct.pack(">i", len(payload))
        msg = header + payload
        
        print(f"Payload length: {len(payload)}")
        print(f"Client version payload: {client_version_payload.hex()}")
        print(f"Name payload: {name_payload.hex()}")

        # Send the login packet
        self.sock.sendall(msg)
        print("Sent PacketLogin with corrected string format!")
        
        # Read response
        hdr = self.sock.recv(6)
        if len(hdr) == 6:
            pkt_id = struct.unpack(">h", hdr[:2])[0]
            pkt_len = struct.unpack(">i", hdr[2:6])[0]
            print("Received packet id:", pkt_id, "payload length:", pkt_len)
            
            if pkt_len > 0:
                data = b''
                while len(data) < pkt_len:
                    chunk = self.sock.recv(pkt_len - len(data))
                    if not chunk:
                        break
                    data += chunk
                print("Login response payload:", data.hex())
            else:
                print("Login successful (no payload)")
                
        # start keep-alive thread
        keep_alive_thread = threading.Thread(
            target=start_keep_alive_loop, 
            args=(self.sock,), 
            daemon=True
        )
        keep_alive_thread.start()
        print("Eternally connected")



#############################################################################
    def send_chat_command(self, command):
        """Send a server command via chat (requires OP permissions first)"""
        message_bytes = command.encode('utf-16be')
        payload = struct.pack(">i", len(command)) + message_bytes
        header = struct.pack(">h", packet_dict["PacketChat"]) + struct.pack(">i", len(payload)) 
        self.sock.sendall(header + payload)
    
    def send_action(self, action_id: int, payload: bytes = b"") -> bytes:
        """
        Sends an action packet to the server.
        action_id: the packet ID for this action
        payload: bytes of the action payload
        Returns the server's response payload (if any)
        """
        header = struct.pack(">hI", action_id, len(payload))
        self.sock.sendall(header + payload)

        # read response header (6 bytes)
        hdr = self.sock.recv(6)
        if len(hdr) < 6:
            return b""

        pkt_id, pkt_len = struct.unpack(">hI", hdr)
        data = b""
        while len(data) < pkt_len:
            chunk = self.sock.recv(pkt_len - len(data))
            if not chunk:
                break
            data += chunk
        return data


    def recv_packet(self):
        hdr = self.sock.recv(6)
        if not hdr:
            return None, None
        pkt_id, pkt_len = struct.unpack(">hI", hdr)
        payload = self.sock.recv(pkt_len) if pkt_len > 0 else b""
        return pkt_id, payload


In [63]:
class MonopolyDealEnv(gym.Env):
    def __init__(self):
        super(MonopolyDealEnv, self).__init__()

        # Connect to server via socket
        HOST = "127.0.0.1"
        PORT = 27599
        PLAYER_NAME = "RLAgent"
        CLIENT_VERSION = "1.0.0"
        PROTOCOL_VERSION = 24

        # Initialize client. Login is part of this.
        self.client = MDClient(HOST, PORT, PLAYER_NAME, CLIENT_VERSION, PROTOCOL_VERSION)
        
        # Define what actions are available
        self.action_space = spaces.Discrete(3)

        self.action_meanings={
            0: "DRAW_CARD",  # PacketActionDraw, ID: 47
            1: "END_TURN",   # PacketActionEndTurn, ID: 48
            2: "DO_NOTHING", # wait
        }
        
        # Define what the agent can observe
        self.observation_space = spaces.Dict({
            "hand": spaces.Box(low=0, high=100, shape=(5,), dtype=int),  
            "board": spaces.Box(low=0, high=100, shape=(10,), dtype=int),  
            "turn": spaces.Discrete(5)  
        })

    def step(self, action):
        # Execute the chosen action
        if action == 0:  # DRAW_CARD
            self.client.send_action(47, b"")  # PacketActionDraw
            print("Action: Drew card")
            
        elif action == 1:  # END_TURN  
            self.client.send_action(48, b"")  # PacketActionEndTurn
            print("Action: Ended turn")
            
        # action == 2 does nothing
        elif action == 2:
            print("Action: Wait")

        else:
            pass
        
        # TODO: Get actual observation from server
        obs = np.zeros(10, dtype=np.float32)
        reward = 0.0
        done = False
        
        return obs, reward, done, False, {}
        
        return obs, reward, done, False, {}

    def reset(self, seed=None, options=None):
        """Reset the game environment and return initial observation"""
        # Send reset command to server
        self.client.send_chat_command("/reset")
        time.sleep(1)  # Wait for reset to complete
        
        # Send start command to begin new game
        self.client.send_chat_command("/start") 
        time.sleep(2)  # Wait for game to start
        
        # Get initial game state (you'll implement this next)
        initial_obs = self._get_observation()
        
        return initial_obs, {}
    
    def close(self):
        self.client.sock.close()


    def _get_observation(self):
        """Get current game state observation"""
        # TODO: Parse actual game state from server packets
        # For now, return placeholder
        return {"hand": [0]*5, "board": [0]*10, "turn": 0}


In [65]:
env = MonopolyDealEnv()
print("environment started")
# I THINK only run this cell ONCE

Sent PacketInitiateLogin(24)
Received packet id: 4 payload length: 24
Payload bytes: b'\x00\x00\x00\x18\x00\x00\x00\x10n/\xd5\x0f\x9e\xb6\xb1\xe7X5l\x1a\x02$\xbc\xab'
Payload length: 56
Client version payload: 000000050031002e0030002e0030
Name payload: 000000070052004c004100670065006e0074
Sent PacketLogin with corrected string format!
Received packet id: 39 payload length: 8
Login response payload: 0000000100000000
Eternally connected
environment started


In [59]:
# obs, info = env.reset()
# print("Game reset complete. Initial observation:\n", obs)

In [60]:
# env.client.send_chat_command(f"/op {env.client.player_name}")
# print("Gave RLAgent op status")

In [66]:
env.client.send_chat_command("/start")
print("Started game")

Started game


In [74]:
# PacketActionDraw ID: 47

# Test drawing a card
obs, reward, done, truncated, info = env.step(0)
print("Draw action sent to server!")

Draw action sent to server!


In [75]:
obs, reward, done, truncated, info = env.step(1)
print("End Turn action sent to server!")

End Turn action sent to server!


In [69]:
# Train without watching - focus on algorithm
# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=10000)  # Train without GUI

In [ ]:
# First make yourself OP, then add bot
# Give yourself OP permissions
# send_chat_command(sock, f"/op {self.player_name}")

In [ ]:
# Add a bot named EasyBot
# send_chat_command(sock, "/addbot Bot1")

In [ ]:
# send_chat_command(sock, "/kick Logan")

In [ ]:
# Send command to start the game
# send_chat_command(sock, "/start")

In [ ]:
# Have your bot check the turn order
# send_chat_command(sock, "/listplayers")

In [ ]:
# send_chat_command(sock, "/nextturn")